In [48]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA

In [6]:
data = pd.read_csv('./data/normalized.csv')

In [16]:
data.head()

,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,Median Income,poverty_rate,share_white,share_black,share_native_american,share_asian,share_hispanic,percent_completed_hs
0,shot,gun,0.012917,M,A,Shelton,WA,True,attack,Not fleeing,False,10.520617,0.286,0.789,0.008,0.037,0.011,0.192,0.801
1,shot,gun,0.008724,M,W,Aloha,OR,False,attack,Not fleeing,False,11.093843,0.149,0.709,0.026,0.010,0.089,0.211,0.881
2,shot,knife,0.014315,F,W,Aloha,OR,True,other,Not fleeing,False,11.093843,0.149,0.709,0.026,0.010,0.089,0.211,0.881
3,shot and Tasered,unarmed,-0.008047,M,H,Wichita,KS,False,other,Not fleeing,False,10.735244,0.173,0.719,0.115,0.012,0.048,0.153,0.875
4,shot,vehicle,-0.005951,M,W,Wichita,KS,False,attack,Car,False,10.735244,0.173,0.719,0.115,0.012,0.048,0.153,0.875


In [35]:
cols = ['manner_of_death', 'armed', 'gender', 'race', 'city', 'state', 'threat_level', 'flee']
encoded = []
for col in cols:
    encoded.append(pd.get_dummies(data[col], prefix=col))
noncat = data.drop(cols, axis=1)
noncat['signs_of_mental_illness'] = pd.factorize(noncat['signs_of_mental_illness'])[0]
noncat['body_camera'] = pd.factorize(noncat['body_camera'])[0]
encoded.append(noncat)
encoded = pd.concat(encoded, axis=1)


In [36]:
encoded['signs_of_mental_illness']

0       0
1       1
2       0
3       1
4       1
       ..
2317    1
2318    1
2319    1
2320    1
2321    1
Name: signs_of_mental_illness, Length: 2322, dtype: int64

In [37]:
x = encoded.drop('signs_of_mental_illness', axis=1)
y = encoded['signs_of_mental_illness']

In [65]:
pca = PCA(n_components=2)
pca_encoded = pca.fit_transform(x)

In [66]:
pca_df = pd.DataFrame(data = pca_encoded
             , columns = ['principal component 1', 'principal component 2'])

In [67]:
pca_df.head()

,principal component 1,principal component 2
0,0.238561,-0.617628
1,-0.335146,-0.832486
2,-0.644716,0.991583
3,-0.237050,1.258673
4,0.044266,-0.168632


In [68]:
pca.explained_variance_ratio_

array([0.14410627, 0.0982367 ])

In [118]:
encoded.to_csv('./data/encoded.csv')

In [74]:
from qiskit import BasicAer
from qiskit.circuit.library import ZZFeatureMap
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import QSVM
from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name

seed = 10599
aqua_globals.random_seed = seed

In [ ]:
total = pd.concat([pca_df, y], axis=1)
x = {'0': total[total['signs_of_mental_illness'] == 0][['principal component 1', 'principal component 2']].to_numpy(),
 '1': total[total['signs_of_mental_illness'] == 1][['principal component 1', 'principal component 2']].to_numpy()}
small_x = {'ill': x['0'], 'not_ill': x['1'][:560]}

In [117]:
feature_dim = 2
feature_map = ZZFeatureMap(feature_dimension=feature_dim, reps=2, entanglement='linear')
qsvm = QSVM(feature_map, small_x, small_x, pca_df.to_numpy())

backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed)

result = qsvm.run(quantum_instance)

print(f'Testing success ratio: {result["testing_accuracy"]}')
print()
print('Prediction from datapoints set:')
print(f'  ground truth: {map_label_to_class_name(datapoints[1], qsvm.label_to_class)}')
print(f'  prediction:   {result["predicted_classes"]}')
predicted_labels = result["predicted_labels"]
print(f'  success rate: {100*np.count_nonzero(predicted_labels == datapoints[1])/len(predicted_labels)}%')

QiskitError: 'Keyboard interrupt in parallel_map.'